In [35]:
import pickle
import numpy as np
import sympy as sym
from sympy.utilities.autowrap import autowrap
sym.init_printing()

In [36]:
def get_symbols():
    alpha, beta, gamma = sym.symbols('alpha beta gamma')  # spacecraft angles
    alphaDot, betaDot, gammaDot = sym.symbols('alphaDot betaDot gammaDot')  # spacecraft velocities
    theta = sym.symbols('theta1:8')  # joint angles assuming 7DoF
    thetaDot = sym.symbols('thetaDot1:8')  # joint velocities
    ang_s = (alpha, beta, gamma)
    omega_s = (alphaDot, betaDot, gammaDot)
    return theta, thetaDot, ang_s, omega_s

In [37]:
# states = Matrix([alpha, beta, gamma, theta_1, theta_2, theta_3, alpha_d, beta_d, gamma_d, theta_1d, theta_2d, theta_3d])
theta, thetaDot, ang_s, omega_s = get_symbols()
# MassMatrix = np.load('MassMatrix.npy', allow_pickle=True)
# Coriolis = np.load('CoriolisVector.npy', allow_pickle=True)

In [38]:
# with open('MassMat_sym.pickle', 'rb') as inM:
#     MassMatrix = pickle.loads(inM.read())

In [39]:
# Mass_C = autowrap(MassMatrix, backend='cython', tempdir='./MassMatrix')
# print('mass written')

In [40]:
# Mass_C = autowrap(MassMatrix, backend='cython', tempdir='./MassMatrix')
# print('mass written')
# Coriolis_C = autowrap(Coriolis, backend='cython', tempdir='./CoriolisVector')
# print('coriolis written')

In [41]:
%load_ext Cython

The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython


In [42]:
%%writefile MassMatrix.pyxbld
#            module name specified by `%%cython_pyximport` magic
#            |        just `modname + ".pyx"`
#            |        |
def make_ext(modname, pyxfilename):
    from setuptools.extension import Extension
    return Extension(modname,
                     sources=[pyxfilename, 'MassMatrix.c'],
                     include_dirs=['.', np.get_include()])

Overwriting MassMatrix.pyxbld


In [43]:
print('ok')

ok


In [47]:
%%cython_pyximport MassMatrix
cimport numpy as cnp # cimport gives us access to NumPy's C API
# here we just replicate the function signature from the header
cdef extern from "MassMatrix.h":
    void MassMatrix(double *y, double *Mass_Matrix)

# here is the "wrapper" signature that conforms to the odeint interface
def MassMatrix(cnp.ndarray[cnp.double_t, ndim=1] y, double t):
    # preallocate our output array
    cdef cnp.ndarray[cnp.double_t, ndim=1] Mass_Matrix = np.empty(100, dtype=np.double)
    # now call the C function
    MassMatrix(<double *> y.data, <double *> Mass_Matrix.data)
    # return the result
    return Mass_Matrix

In [50]:
rnd = np.random.rand(10)
rnd
MassMatrix(rnd, 1)

array([0.17796635, 0.51934012, 0.30273919, 0.73118138, 0.0037315 ,
       0.99020296, 0.94158919, 0.99545889, 0.3583178 , 0.6668142 ])